In [2]:
pip install streamlit-folium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.6/524.6 kB 867.5 kB/s  0:00:00:00:01

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import streamlit as st
import geopandas as gpd
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from streamlit_folium import st_folium

st.set_page_config(layout="wide", page_title="Carte médecins")

# -----------------------------
# 1. Chargement des données
# -----------------------------
@st.cache_data
def load_data():
    clusters = gpd.read_file("FV_medecins_clusters_viz.gpkg")
    med = pd.read_csv("offre_medical.csv")
    # convertir médecins en GeoDataFrame
    med_gdf = gpd.GeoDataFrame(
        med,
        geometry=gpd.points_from_xy(med.longitude, med.latitude),
        crs="EPSG:4326"
    )
    return clusters, med_gdf

clusters, med_gdf = load_data()

# -----------------------------
# 2. Menu spécialité
# -----------------------------
specialites = sorted(clusters["Libellé savoir-faire"].dropna().unique())
choix_specialite = st.selectbox("Choisir une spécialité :", specialites)

# Filtrer clusters et médecins
clusters_sel = clusters[clusters["Libellé savoir-faire"] == choix_specialite]
med_sel = med_gdf[med_gdf["Libellé savoir-faire"] == choix_specialite]

# -----------------------------
# 3. Carte Folium
# -----------------------------
m = folium.Map(location=[46.6, 2.5], zoom_start=6, tiles="cartodbpositron")

# --- Ajouter clusters ---
min_c = clusters_sel["med_count"].min()
max_c = clusters_sel["med_count"].max()

def color_scale(v):
    if pd.isna(v):
        v = 0
    ratio = (v - min_c) / (max_c - min_c + 1e-5)
    r = int(255 * ratio)
    g = int(255 * (1 - ratio))
    return f"#{r:02x}{g:02x}00"

for _, row in clusters_sel.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=8,
        fill=True,
        fill_opacity=0.7,
        color=color_scale(row["med_count"]),
        popup=folium.Popup(
            f"<b>Cluster:</b> {row.get('cluster_id', 'N/A')}<br>"
            f"<b>Médecins:</b> {row.get('med_count',0)}<br>"
            f"<b>Spécialité:</b> {row.get('specialite')}",
            max_width=250
        )
    ).add_to(m)

# --- Ajouter points exacts des médecins ---
marker_cluster = MarkerCluster(name="Médecins").add_to(m)

for _, row in med_sel.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=4,
        color="blue",
        fill=True,
        fill_opacity=0.9,
        popup=folium.Popup(
            f"<b>Médecin:</b> {row.get('Libellé savoir-faire')}<br>"
            f"<b>Adresse:</b> {row.get('result_city','')}",
            max_width=250
        )
    ).add_to(marker_cluster)

# Légende
folium.LayerControl().add_to(m)

# -----------------------------
# 4. Affichage Streamlit
# -----------------------------
st_folium(m, width=1200, height=800)

2025-11-20 15:38:54.289 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:38:54.298 No runtime found, using MemoryCacheStorageManager
2025-11-20 15:38:56.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:38:56.082 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:38:56.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:38:56.091 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:38:56.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:38:56.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-20 15:40:54.128 Thread 'MainThread':

{'last_clicked': None,
 'last_object_clicked': None,
 'last_object_clicked_tooltip': None,
 'last_object_clicked_popup': None,
 'all_drawings': None,
 'last_active_drawing': None,
 'bounds': {'_southWest': {'lat': -21.376461, 'lng': -151.46},
  '_northEast': {'lat': 51.048278, 'lng': 55.712639}},
 'zoom': 6,
 'last_circle_radius': None,
 'last_circle_polygon': None,
 'selected_layers': None}

In [6]:
import geopandas as gpd

# Charger le fichier GeoPackage
gdf = gpd.read_file("FV_medecins_clusters_viz.gpkg")

# Afficher les 5 premières lignes
print(gdf.head())

                Libellé savoir-faire  cluster_id  med_count  longitude  \
0    Pédiatrie & chirurgie infantile        54.0      586.0 -61.252531   
1                      Ophtalmologie        54.0      586.0 -61.536929   
2  Médecine physique et réadaptation        54.0      586.0 -61.500251   
3    Psychiatrie & neuro-psychiatrie        54.0      586.0 -61.725021   
4                      Ophtalmologie        54.0      586.0 -61.731941   

    latitude                    geometry  
0  15.978380  POINT (-61.25253 15.97838)  
1  16.237094  POINT (-61.53693 16.23709)  
2  16.265018  POINT (-61.50025 16.26502)  
3  16.001806  POINT (-61.72502 16.00181)  
4  15.997978  POINT (-61.73194 15.99798)  


In [7]:
import pandas as pd

# Charger le fichier CSV
df = pd.read_csv("offre_medical.csv")

# Afficher les 5 premières lignes
print(df.head())

   __id  Identifiant PP  Identification nationale PP  Code profession  \
0  32.0    1.000001e+10                 8.100000e+11             10.0   
1  47.0    1.000001e+10                 8.100000e+11             10.0   
2  77.0    1.000001e+10                 8.100000e+11             10.0   
3  79.0    1.000001e+10                 8.100000e+11             10.0   
4  84.0    1.000002e+10                 8.100000e+11             10.0   

  Libellé profession Code savoir-faire               Libellé savoir-faire  \
0            Médecin              SM40    Pédiatrie & chirurgie infantile   
1            Médecin              SM38                      Ophtalmologie   
2            Médecin              SM29  Médecine physique et réadaptation   
3            Médecin              SM42    Psychiatrie & neuro-psychiatrie   
4            Médecin              SM38                      Ophtalmologie   

  Numéro FINESS site Numéro FINESS établissement juridique  Num_voie  ...  \
0          970108874 

In [1]:
import os
os.path.exists("notebooks/Quentin/FV_medecins_clusters_viz.gpkg")

False